In [1]:
import json
import time
import random
import pymongo
import copy
import MyCommon
from pymongo import MongoClient
from collections import deque
import cassiopeia
from cassiopeia import riotapi
import numpy as np
from sklearn import preprocessing, cluster, decomposition
from scipy.cluster.vq import kmeans,vq
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.decomposition import FastICA, PCA
from sklearn import metrics
import matplotlib.pyplot as plt

In [4]:
from openpyxl import load_workbook
spring_2016 = load_workbook('./2016Data/NA-EU-LCK-LMS-2016-spring-match-data-v1-OraclesElixir.xlsx')

In [5]:
spring_2016_data = spring_2016['data']

In [34]:
for row in range(1, 2):
    for col in range(1,81):
        print col
        cell = spring_2016_data.cell(row=row, column=col)
        print cell.value
        cell = spring_2016_data.cell(row=row + 1, column=col)
        print cell.value
    break

1
gameid
1001470051
2
url
http://matchhistory.na.leagueoflegends.com/en/#match-details/TRLH3/1001470051?gameHash=4c75eed3e321fae9&tab=overview
3
league
EULCS
4
split
2016-1
5
date
2016-01-14 11:58:56
6
week
1
7
game
1
8
patchno
6.01
9
playerid
1
10
side
Blue
11
position
Top
12
player
Gamsu
13
team
Fnatic
14
champion
Olaf
15
ban1
Kindred
16
ban2
Kassadin
17
ban3
Tahm Kench
18
gamelength
35.95
19
result
1
20
k
9
21
d
1
22
a
3
23
teamkills
22
24
teamdeaths
8
25
fb
0
26
fbvictim
0
27
fbtime
None
28
kpm
0.250347705146
29
okpm
0.0278164116829
30
ckpm
0.834492350487
31
fd
0
32
fdtime
10.4983
33
teamdragkills
3
34
oppdragkills
1
35
herald
1
36
heraldtime

37
ft
1
38
fttime
15.8962166667
39
firsttothreetowers
1
40
teamtowerkills
11
41
opptowerkills
0
42
fbaron
1
43
fbarontime
30.4260166667
44
teambaronkills
1
45
oppbaronkills
0
46
dmgtochamps
22264
47
dmgtochampsperminute
619.304589708
48
dmgshare
0.252271851701
49
earnedgoldshare
0.225269070642
50
wards
18
51
wpm
0.500695410292
52
wardshare
0.

In [24]:
COL_NUM = spring_2016_data.max_column
ROW_NUM = spring_2016_data.max_row
print COL_NUM
print ROW_NUM

99
6949


In [36]:
#league
LEAGUE_COL = 3
STARTING_ROW = 2
league_set = set(['EULCS', 'NALCS', 'LMS', 'LCK'])
lck_num = 0
for row in range(STARTING_ROW,ROW_NUM):
    a_league = spring_2016_data.cell(row = row, column = LEAGUE_COL)
    if a_league.value == 'LCK':
        lck_num += 1
LCK_GAME_NUM = lck_num/12

In [38]:
#lck team
TEAM_COL = 13
TEAM_GAP = 12
BLUE_START = 0
RED_START = 5
team_set = set()
lck_starting_row = 0
for row in range(STARTING_ROW, ROW_NUM):
    a_league = spring_2016_data.cell(row = row, column = LEAGUE_COL)
    if lck_starting_row == 0 and a_league.value == 'LCK':
        lck_starting_row = row
        
for game_idx in range(LCK_GAME_NUM):
    blue_team = spring_2016_data.cell(row = lck_starting_row + game_idx * TEAM_GAP + BLUE_START,
                                      column = TEAM_COL)
    red_team = spring_2016_data.cell(row = lck_starting_row + game_idx * TEAM_GAP + RED_START,
                                      column = TEAM_COL)
    team_set.add(blue_team.value)
    team_set.add(red_team.value)
print team_set

set([u'Jin Air Green Wings', u'SBENU Sonicboom', u'KT Rolster', u'ROX Tigers', u'Samsung Galaxy', u'Longzhu Gaming', u'SK Telecom T1', u'CJ Entus', u'Kongdoo Monster', u'Afreeca Freecs'])


In [45]:
#members
MEMBER_COL = 12
teams_members = {}
for a_team in team_set:
    teams_members[a_team] = set()
    
for game_idx in range(LCK_GAME_NUM):
    for pos_idx in range(10):
        a_member = spring_2016_data.cell(row = lck_starting_row + game_idx * TEAM_GAP + pos_idx,
                                      column = MEMBER_COL)
        a_team = spring_2016_data.cell(row = lck_starting_row + game_idx * TEAM_GAP + pos_idx,
                                      column = TEAM_COL)
        teams_members[a_team.value].add(a_member.value)

for a_key in teams_members:
    print a_key
    print teams_members[a_key]

Jin Air Green Wings
set([u'TrAce', u'SoHwan', u'Winged', u'Kuzan', u'Pilot', u'Chei'])
SBENU Sonicboom
set([u'SaSin', u'Nuclear', u'Soul', u'Flawless', u'Secret', u'SoaR'])
KT Rolster
set([u'Fly', u'Score', u'Arrow', u'Hachani', u'IgNar', u'Ssumday'])
ROX Tigers
set([u'PraY', u'Peanut', u'GorillA', u'Smeb', u'Kuro'])
Samsung Galaxy
set([u'CoreJJ', u'Stitch', u'CuVee', u'Crown', u'Ambition', u'Wraith'])
Longzhu Gaming
set([u'Cpt Jack', u'Crash', u'Expession', u'Frozen', u'Zzus', u'Fury', u'Flame', u'Pure', u'Coco', u'Chaser'])
SK Telecom T1
set([u'Duke', u'Faker', u'Bang', u'Scout', u'Wolf', u'Blank', u'Bengi'])
CJ Entus
set([u'Bdd', u'Kramer', u'Bubbling', u'Sky', u'Untara', u'MadLife'])
Kongdoo Monster
set([u'Hipo', u'GuGer', u'Crush', u'Edge', u'Roach', u'SSol'])
Afreeca Freecs
set([u'Mickey', u'lira', u'SnowFlower', u'Sangyoon', u'ikssu'])
